### My Data Science Clustering Project

#### A look at the Cities of Australia to determine where to start a new housing project
##### By K. Feliciano

#### Introduction:

   As an investor living in Australia; I am trying to look at the different Cities in Australia to figure out where to start a new housing project.
   The analysis of the data will tell me if a city's amenities drives the housing prices for potential buyers once the project is set up.

#### The Data and Methodology

I will be using the FourSquare data to gather insights on the venues in the different cities to get an idea on what amenities are available for each cities.

In [3]:
# Importing libraries
!conda install -c conda-forge beautifulsoup4 --yes # install BeautifulSoup 4
from bs4 import BeautifulSoup
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
beautifulsoup4            4.6.3                    py35_0    conda-forge
Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
geopy                     1.18.1                     py_0    conda-forge
Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge
Libraries imported.


#### Scrape a web URL to get average housing prices data

In [12]:
# Page URL
url = 'https://www.livingin-australia.com/australian-house-prices/'

# assign the result of a request to a variable
page = requests.get(url)

from bs4 import BeautifulSoup
soup = BeautifulSoup(page.text, 'html.parser')

# find the table object
table_content = soup.find(id="oztable")

#find all the rows
tb_rows = table_content.find_all("tr")

data = []  

# insert the table data in a list object
for row in tb_rows:
    cols = row.find_all('td')
    
    # insert data to list if its not emty
    if len(cols) != 0:
        cols = [ele.text.strip() for ele in cols]
        
        # insert only if 2nd element of list (Borough) is not equal to Not Assigned
        if cols[1] != 'Not assigned':
            data.append([ele for ele in cols if ele])
            
# define the dataframe columns
column_names = ['City', 'AverageHousePrice', 'PctChange'] 

# instantiate the dataframe
My_DF = pd.DataFrame(data, columns=column_names)
My_DF

,City,AverageHousePrice,PctChange
0,"Sydney, NSW","$852,000",+ 15.5 %
1,"Melbourne, Vic","$641,200",+ 13.7 %
2,"Brisbane, Qld","$486,000",+ 3.6 %
3,"Adelaide, SA","$425,000",+ 4.2 %
4,"Perth, WA","$490,000",– 0.6 %
5,"Hobart, Tas","$345,000",+ 11.2 %
6,"Canberra, ACT","$595,000",+ 9.3 %


In [13]:
geo_loc = pd.DataFrame(columns = ['City','Latitude','Longitude'])
geo_loc

,City,Latitude,Longitude


In [16]:
for place in My_DF['City']:
    geolocator = Nominatim()
    place_tmp = place + ' Australia'
    location = geolocator.geocode(place_tmp,timeout=10)
    latitude = location.latitude
    longitude = location.longitude
    print('The geograpical coordinate of {} are {}, {}.'.format(place,latitude, longitude))
    geo_loc = geo_loc.append({'City': place,
        'Latitude' : latitude,
        'Longitude' : longitude}, ignore_index = True)

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The geograpical coordinate of Sydney, NSW are -33.8548157, 151.2164539.
The geograpical coordinate of Melbourne, Vic are -37.8142176, 144.9631608.
The geograpical coordinate of Brisbane, Qld are -27.4689682, 153.0234991.
The geograpical coordinate of Adelaide, SA are -34.9281805, 138.5999312.
The geograpical coordinate of Perth, WA are -31.9527121, 115.8604796.
The geograpical coordinate of Hobart, Tas are -42.8825088, 147.3281233.
The geograpical coordinate of Canberra, ACT are -35.2975906, 149.1012676.


In [22]:
# Combine the Geo Coordinate Dataframe to the earlier Dataframe
Merged_DF = pd.merge(left=My_DF,right=geo_loc, left_on='City', right_on='City')
Merged_DF = Merged_DF.drop('latitude', axis=1)
Merged_DF = Merged_DF.drop('longitude', axis=1)
Merged_DF = Merged_DF.drop('region', axis=1)
Merged_DF

,City,AverageHousePrice,PctChange,Latitude,Longitude
0,"Sydney, NSW","$852,000",+ 15.5 %,-33.8548,151.216
1,"Melbourne, Vic","$641,200",+ 13.7 %,-37.8142,144.963
2,"Brisbane, Qld","$486,000",+ 3.6 %,-27.469,153.023
3,"Adelaide, SA","$425,000",+ 4.2 %,-34.9282,138.6
4,"Perth, WA","$490,000",– 0.6 %,-31.9527,115.86
5,"Hobart, Tas","$345,000",+ 11.2 %,-42.8825,147.328
6,"Canberra, ACT","$595,000",+ 9.3 %,-35.2976,149.101


#### Show the Map of Australia

In [27]:
# create map of Singapore using latitude and longitude values
map_AU = folium.Map(location=[latitude, longitude], zoom_start=4)

# add markers to map
for lat, lng, label in zip(Merged_DF['Latitude'], Merged_DF['Longitude'], Merged_DF['City']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_AU)  
    
map_AU

#### Get nearby venues in the different cities

In [41]:
CLIENT_ID = 'XNRZF4NPZCEGYZVHFIU43YCB4O1PVQPLX5YJTPD1BVRJ4TLZ' 
CLIENT_SECRET = '3BD1AMG2OGWRZD2DUKUDNKQ41DIV2FBAOBPAQ0Q5MZIVCAEN'
VERSION = '20190103' # Foursquare API version

In [48]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 
                  'City Latitude', 
                  'City Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

##### Insert venues in dataframe

In [49]:
Aus_Venues = getNearbyVenues(names=Merged_DF['City'],
                                   latitudes=Merged_DF['Latitude'],
                                   longitudes=Merged_DF['Longitude']
                                  )

Sydney, NSW
Melbourne, Vic
Brisbane, Qld
Adelaide, SA
Perth, WA
Hobart, Tas
Canberra, ACT


#### View the venues

In [50]:
print(Aus_Venues.shape)
Aus_Venues.head()

(481, 7)


,City,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Sydney, NSW",-33.854816,151.216454,Sydney Opera House - Concert Hall,-33.856595,151.215058,Concert Hall
1,"Sydney, NSW",-33.854816,151.216454,Drama Theatre,-33.856863,151.214759,Theater
2,"Sydney, NSW",-33.854816,151.216454,Sydney Opera House,-33.857260,151.215040,Opera House
3,"Sydney, NSW",-33.854816,151.216454,Sydney Opera House - Studio,-33.856991,151.214482,Theater
4,"Sydney, NSW",-33.854816,151.216454,Sydney Opera House - Playhouse,-33.857518,151.214559,Theater


In [52]:
# Get the number of Venue Category

Aus_Venues.groupby('City').count()
print('There are {} uniques categories.'.format(len(Aus_Venues['Venue Category'].unique())))

There are 129 uniques categories.


#### Venue Data

In [55]:
# one hot encoding
Aus_onehot = pd.get_dummies(Aus_Venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Aus_onehot['City'] = Aus_Venues['City'] 

# move neighborhood column to the first column
fixed_columns = [Aus_onehot.columns[-1]] + list(Aus_onehot.columns[:-1])
Aus_onehot = Aus_onehot[fixed_columns]

print(Aus_onehot.shape)
Aus_grouped = Aus_onehot.groupby('City').mean().reset_index()
print(Aus_grouped.shape)

(481, 130)
(7, 130)


#### City's top 5 venues

In [56]:
num_top_venues = 3

for hood in Aus_grouped['City']:
    print("----"+hood+"----")
    temp = Aus_grouped[Aus_grouped['City'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, SA----
              venue  freq
0       Coffee Shop  0.09
1  Asian Restaurant  0.08
2              Café  0.08


----Brisbane, Qld----
         venue  freq
0         Café  0.14
1  Coffee Shop  0.06
2        Hotel  0.05


----Canberra, ACT----
                venue  freq
0               Beach  0.33
1  Athletics & Sports  0.33
2     Harbor / Marina  0.33


----Hobart, Tas----
         venue  freq
0         Café  0.14
1  Coffee Shop  0.11
2        Hotel  0.06


----Melbourne, Vic----
         venue  freq
0  Coffee Shop  0.08
1          Bar  0.08
2         Café  0.08


----Perth, WA----
         venue  freq
0  Coffee Shop  0.08
1          Bar  0.06
2         Café  0.05


----Sydney, NSW----
          venue  freq
0       Theater  0.21
1  Concert Hall  0.14
2  Cocktail Bar  0.14




#### Insert venues to a data frame

In [57]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [58]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['City']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['City'] = Aus_grouped['City']

for ind in np.arange(Aus_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Aus_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, SA",Coffee Shop,Asian Restaurant,Café,Chinese Restaurant,Pub,Italian Restaurant,Bar,Hotel,Ice Cream Shop,Sushi Restaurant
1,"Brisbane, Qld",Café,Coffee Shop,Hotel,Korean Restaurant,Sushi Restaurant,Burger Joint,Japanese Restaurant,Chinese Restaurant,Chocolate Shop,Malay Restaurant
2,"Canberra, ACT",Harbor / Marina,Athletics & Sports,Beach,Dim Sum Restaurant,Electronics Store,Dumpling Restaurant,Donut Shop,Dive Bar,Distillery,Dessert Shop
3,"Hobart, Tas",Café,Coffee Shop,Bakery,Hotel,Indian Restaurant,Asian Restaurant,Rental Car Location,Pub,Park,Juice Bar
4,"Melbourne, Vic",Coffee Shop,Bar,Café,Cocktail Bar,Dessert Shop,Shopping Mall,Chocolate Shop,Candy Store,Cosmetics Shop,Pizza Place
5,"Perth, WA",Coffee Shop,Bar,Café,Hotel,Korean Restaurant,Australian Restaurant,Vietnamese Restaurant,Wine Bar,Shopping Mall,Grocery Store
6,"Sydney, NSW",Theater,Concert Hall,Cocktail Bar,Coffee Shop,Park,Tree,Opera House,Australian Restaurant,Plaza,Café


In [60]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

kclusters = 3

Aus_grouped_clustering = Aus_grouped.drop('City', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Aus_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 1, 0, 0, 0, 2], dtype=int32)

In [94]:
Aus_merged = Merged_DF

Aus_merged = Aus_merged.join(neighborhoods_venues_sorted.set_index('City'), on='City')

Aus_merged['Cluster Labels'] = kmeans.labels_

Aus_merged

,City,AverageHousePrice,PctChange,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
0,"Sydney, NSW","$852,000",+ 15.5 %,-33.8548,151.216,Theater,Concert Hall,Cocktail Bar,Coffee Shop,Park,Tree,Opera House,Australian Restaurant,Plaza,Café,0
1,"Melbourne, Vic","$641,200",+ 13.7 %,-37.8142,144.963,Coffee Shop,Bar,Café,Cocktail Bar,Dessert Shop,Shopping Mall,Chocolate Shop,Candy Store,Cosmetics Shop,Pizza Place,0
2,"Brisbane, Qld","$486,000",+ 3.6 %,-27.469,153.023,Café,Coffee Shop,Hotel,Korean Restaurant,Sushi Restaurant,Burger Joint,Japanese Restaurant,Chinese Restaurant,Chocolate Shop,Malay Restaurant,1
3,"Adelaide, SA","$425,000",+ 4.2 %,-34.9282,138.6,Coffee Shop,Asian Restaurant,Café,Chinese Restaurant,Pub,Italian Restaurant,Bar,Hotel,Ice Cream Shop,Sushi Restaurant,0
4,"Perth, WA","$490,000",– 0.6 %,-31.9527,115.86,Coffee Shop,Bar,Café,Hotel,Korean Restaurant,Australian Restaurant,Vietnamese Restaurant,Wine Bar,Shopping Mall,Grocery Store,0
5,"Hobart, Tas","$345,000",+ 11.2 %,-42.8825,147.328,Café,Coffee Shop,Bakery,Hotel,Indian Restaurant,Asian Restaurant,Rental Car Location,Pub,Park,Juice Bar,0
6,"Canberra, ACT","$595,000",+ 9.3 %,-35.2976,149.101,Harbor / Marina,Athletics & Sports,Beach,Dim Sum Restaurant,Electronics Store,Dumpling Restaurant,Donut Shop,Dive Bar,Distillery,Dessert Shop,2


##### View the cluster maps

In [69]:
# create map
map_clusters = folium.Map(location=[latitude2, longitude2], zoom_start=4)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Aus_merged['Latitude'], Aus_merged['Longitude'], Aus_merged['City'], Aus_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [115]:
Aus_merged['AverageHousePrice'] = Aus_merged['AverageHousePrice'].astype(str).replace(',','', regex = True)
Aus_merged['AverageHousePrice'] = Aus_merged['AverageHousePrice'].astype(str).replace('$','', regex = True).astype(float)

Aus_merged['AverageHousePrice'].astype(str).astype(float)
Aus_merged.groupby('Cluster Labels').mean()['AverageHousePrice']



Cluster Labels
0    550640.0
1    486000.0
2    595000.0
Name: AverageHousePrice, dtype: float64